In [1]:
#import
#torch lib
import torch
#neural network
import torch.nn as nn
#optimizer ex. momentum,SGD,RMSprop,Adagrad,Adadelta,Adam
from torch import optim
#loss function ex. softmax *** Activation func?
import torch.nn.functional as F
#others
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
#check cuda is available or not
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

In [3]:
#dataset download from http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html
#datasets file path
lines_filepath = os.path.join("cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus", "movie_conversations.txt")

In [4]:
#Visualize some lines
with open(lines_filepath, "r", encoding="iso-8859-1") as file:
    lines = file.readlines()
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [5]:
#Split each line of the file into a dictionary of fields (lineID, characterID, movieID, character, text)
line_fields = ["lineID", "characterID", "movieID", "character", "text"]
lines = {}
with open(lines_filepath, "r", encoding="iso-8859-1") as f:
    for line in f:
        values = line.split(" +++$+++ ")
        #Extract fields
        lineObj = {}
        if len(values)==5:
            for i, field in enumerate(line_fields):
                lineObj[field] = values[i]
            lines[lineObj["lineID"]] = lineObj

In [7]:
conv_fields = ["character1ID", "character2ID", "movieID", "utteranceIDs"]
conversations = []
with open(conv_filepath, "r", encoding="iso-8859-1") as f:
    for line in f:
        values = line.split(" +++$+++ ")
        #Extract fields
        convObj = {}
        for i, field in enumerate(conv_fields):
            convObj[field] = values[i]
        #convert string type array data to real array
        lineIds = eval(convObj["utteranceIDs"])
        #query lines's data then append to convObj["lines"]
        convObj["lines"] = []
        for lineId in lineIds:
            convObj["lines"].append(lines[lineId])
        conversations.append(convObj)
#Done for extract and combine data to an array...